In [20]:
import gradio as gr

In [21]:
import pandas as pd
import json
import numpy as np
import spacy
import string
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")
from spacy import tokenizer
from spacy.lang.en import English
from nltk.tokenize import sent_tokenize
import re
from bs4 import BeautifulSoup
import unicodedata
import pickle

In [22]:
# https://towardsdatascience.com/text-normalization-for-natural-language-processing-nlp-70a314bfa646
# Not as efficient as other specialized packets... bu at least it works well
contractions_dict = {"ain't": "are not", "'s":" ", " s ":" is", "aren't": "are not", "Aren't": "are not", "Can't": "can not", "can't": "can not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "don t": "do not", "dont": "do not", "dunno": "do not know", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'd've": "he would have", "he'll": "he will", " hes ": " he is ", "he'll've": "he will have", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "i'll": "i will", "'ll":" will", "I'll've": "I will have", "I’m": "I am", "i'm": "i am", "'m": " am", " im ": " i am ", "I've": "I have", "i've": "i have", "I've": "I have", "havent": "have not", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll’ve": "it will have", "let's": "let us", "lets": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't’ve": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "She'll": "she will", "she'll": "she will", "she'll've": "she will have", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have", "that'd": "that would", "that'd've": "that would have", "there'd": "there would", "there'd've": "there would have", "they'd": "they would", "they'd've": "they would have","they'll": "they will", "wont": "will not",
  "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not","what'll": "what will", "what'll've": "what will have", "what're": "what are", "what've": "what have", "when've": "when have", "where’d": "where did", "where've": "where have",
  "who'll": "who will", "who'll've": "who will have", "who've": "who have", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y’all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "You'll": "you will", "you'll've": "you will have", "You're": "you are", "you're": "you are", "you've": "you have", "wanna": "want to", " u ": " you ", " r ": " are ", "gawd": "god", "urlLink": "", "luv": "love", "Luv": "Love", "wuts": "what is", "wasnt": "was not", "Wasnt": "was not", " bday ": "birthday", " gotta ": "got to", "gonna": "going to",
  "nvr": "never", " cuz ": " because ", " cos ": " because "," cant ": " can not ", "yr": " years old", "i\'d": "i\ would", "they\'d": "they\ would", "we\'re": "we\ are", "shouldn\'t": "should\ not", "Don\'t": "Do\ not", "won\'t": "will\ not", "haven\'t": "have\ not", "you\'re": "you\ are", "they\'re": "they\ are", "Didn\'t": "Did\ not", "Hasn\'t": "Has\ not", "I\'d": "I\ would", "you\'ve": "you\ have", "peeps": "people", "it's": "it is", "kinda": "kind of", "buyin": "buying", "Its": "It is", "bout": "about", " ppl ": " people ", " n ": " and ", "enuf": "enough", "btw": "by the way", "BTW": "BY THE WAY", 'b/c': "because", " aabout": "about", " aaabout": "about", "aaaabout": "about"}

In [23]:
contractions_re = re.compile('(%s)'%'|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, s)
def lowering(text):
    return text.lower()
def remove_url(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)
def remove_html_tags_func(text):
    return BeautifulSoup(text, 'html.parser').get_text()
def remove_accents(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
def remove_punctuation(text):
    # return re.sub(r'[^a-zA-Z0-9]', ' ', text) # --> if you allow for numbers
    return re.sub(r'[^a-zA-Z]', ' ', text) # --> if you do not allow for numbers
def remove_extra_spaces(text):
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()

en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words

def delete_stopwords(text):
    words = [word for word in text.split() if word.lower() not in sw_spacy]
    new_text = " ".join(words)
    return new_text

def lemmatizer(sentence):
    doc = nlp(sentence)

    # Lemmatize:
    mytokens = [word.lemma_.strip() for word in doc]

    # Re-join
    sentence = " ".join(mytokens)

    # Return the sentence
    return(sentence)

In [24]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2') 

In [25]:
#Funzione che faccia il preprocessing
def preprocessing(post):
    post = expand_contractions(post)
    post = lowering(post)
    post = remove_url(post)
    post = remove_html_tags_func(post)
    post  = remove_accents(post)
    post = remove_punctuation(post)
    post = remove_extra_spaces(post)
    post = delete_stopwords(post)
    post = model.encode(post)
    return post

In [26]:
preprocessing("Hellooo, time can't wait to see you at https://example.com!             a   of ")

array([-1.05435029e-01, -2.22503748e-02,  2.27699112e-02,  5.09012267e-02,
       -1.88462678e-02, -7.90636167e-02,  3.55912708e-02,  3.37074287e-02,
        1.90408658e-02,  7.16680335e-03, -5.40326349e-03, -4.87990417e-02,
       -4.49391641e-02, -2.36380640e-02,  3.21930088e-02,  9.02613103e-02,
        1.11137420e-01, -5.75760426e-03, -1.16802290e-01,  4.15906534e-02,
        3.46617177e-02,  9.92389116e-03, -9.17087644e-02,  7.94417411e-03,
       -7.40805268e-02, -2.87805796e-02,  4.86160330e-02,  4.37728763e-02,
        3.21470872e-02,  4.92233783e-03, -4.35721464e-02, -3.20933782e-03,
        4.92340326e-02,  2.96427067e-02,  5.86210284e-03,  3.44268866e-02,
       -1.69891305e-02, -1.04742073e-01,  6.85539097e-02,  2.05690470e-02,
        4.80734222e-02, -2.10309215e-02, -3.52458693e-02,  3.16280164e-02,
        4.90955785e-02, -3.02880257e-02,  5.27979247e-02,  1.33442041e-02,
        6.06348626e-02, -1.81277599e-02, -1.25266239e-01, -3.14575657e-02,
       -2.93194558e-02,  

In [12]:
preprocessing("I was walking down the street and met my beautiful boyfriend")

array([-1.26654357e-02, -1.07957739e-02,  4.44977209e-02,  7.04806596e-02,
       -1.79892890e-02,  1.78552475e-02,  5.24795912e-02, -3.97726297e-02,
        1.98965631e-02, -1.01639830e-01,  2.56993044e-02, -2.33228095e-02,
       -4.96508889e-02,  5.13850786e-02,  5.44713810e-02,  6.18546568e-02,
        2.57310290e-02, -1.19378120e-02,  2.73926370e-02,  2.17580646e-02,
       -7.31968656e-02,  5.76196499e-02, -2.60549467e-02,  3.85884568e-02,
       -6.69588447e-02,  4.64364886e-02,  1.19320065e-01,  2.80162022e-02,
        1.05834408e-02,  5.72161153e-02,  9.33581665e-02,  2.24274211e-02,
        1.08116902e-02,  2.92156357e-02,  1.04394266e-02,  6.17729314e-03,
        2.02203859e-02, -1.19530410e-02,  3.26195508e-02, -9.22991522e-03,
       -3.55683528e-02,  2.78363824e-02,  4.48241308e-02,  5.47719672e-02,
        7.58266971e-02, -1.54657140e-01,  3.02674007e-02, -4.92230579e-02,
       -1.03764050e-02, -2.04386488e-02, -7.14077055e-02, -1.39406323e-02,
       -5.24549559e-02, -

In [27]:
train = pd.read_pickle('embedding_100k_prova2.pkl')

In [28]:
train['age_class'] = pd.cut(
        train["age"],
        bins=[12, 18, 28, 50],
        labels=[0, 1, 2]
    ).astype("int")

In [29]:
train

,post,age,gender,length,lemmatize,embedding,age_class
0,ooh shiny new commenting,16,female,4,ooh shiny new commenting,"[-0.18770814, -0.13998552, 0.019073976, 0.0225...",0
1,today parade suked bad band year battle today ...,14,male,48,today parade suke bad band year battle today k...,"[-0.11394925, 0.02999169, 0.014715457, -0.0732...",0
2,know aabout anymore concerned everyday want bo...,24,female,80,know aabout anymore concerned everyday want bo...,"[-0.10992989, -0.04249898, 0.019761039, 0.0499...",1
3,roof sunset posted paul,24,male,7,roof sunset post paul,"[-0.046943568, 0.13227944, 0.02199171, 0.00461...",1
4,god love nanny absolutely greatest woman earth...,23,female,730,god love nanny absolutely great woman earth kn...,"[-0.03844242, -0.08146381, 0.017230963, 0.0577...",1
...,...,...,...,...,...,...,...
104581,w y online gaming squared time try going want ...,24,male,61,w y online gaming square time try going want g...,"[-0.022314172, -0.12119941, -0.022755269, -0.0...",1
104582,jeremy concert high school,41,female,9,jeremy concert high school,"[0.0012280531, 0.0471571, 0.042628232, -0.0627...",2
104583,hey great weekend went laura fun sunday sat da...,17,male,126,hey great weekend go laura fun sunday sat day ...,"[-0.050709814, -0.055519138, 0.07261971, 0.012...",0
104584,house sand fog behrani family died daughter ma...,25,female,115,house sand fog behrani family die daughter mar...,"[-0.02100826, 0.08574633, -0.023208255, -0.004...",1


In [30]:
X = train['embedding'].to_list() # ,'has_young_word', 'word_count', 'has_emoticon', 'has_punctuation', 'has_URL'
# X = pd.DataFrame(embedding['embedding'].to_list(), columns=[f'embedding_{i+1}' for i in range(384)])
# X = pd.concat([X, features], axis = 1)
y = train['gender'].to_list()

In [31]:
# Devo creare dove allenare il modello
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, stratify = y) 

In [32]:
#Il modello che usiamo
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
#Alleno il modello
lr.fit(X_train, y_train)

LogisticRegression()

In [33]:
#Il prdict del modello
from sklearn import metrics
predicted = lr.predict(X_val) #questo mi servirà dentro la funzione da chiamare

In [34]:
X_age = train['embedding'].to_list() # ,'has_young_word', 'word_count', 'has_emoticon', 'has_punctuation', 'has_URL'
# X = pd.DataFrame(embedding['embedding'].to_list(), columns=[f'embedding_{i+1}' for i in range(384)])
# X = pd.concat([X, features], axis = 1)
y_age = train['age_class'].to_list()
X_train_age, X_val_age, y_train_age, y_val_age = train_test_split(X_age, y_age, test_size = 0.2, stratify = y_age) # stratification on the y to avoid class imbalancement
model_age = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model_age.fit(X_train_age, y_train_age)

LogisticRegression(multi_class='multinomial')

In [35]:
#la funzione dentro Gradio
def profAiling(post):
    post = preprocessing(post).reshape(1,-1)
    #print(post)
    predicted = lr.predict_proba(post).tolist()[0]
    class_names = ["Female", "Male"]
    return {class_names[i]: predicted[i] for i in range(2)}

In [36]:
def profAiling_age(post):
    post = preprocessing(post).reshape(1,-1)
    #print(post)
    predicted = model_age.predict_proba(post).tolist()[0]
    class_names = ["Teens", "Twenties", "Thirtis"]
    return {class_names[i]: predicted[i] for i in range(3)}

In [79]:
profAiling_age("LOve LOL shopping mum")

{'Teens': 0.6346174758184201,
 'Twenties': 0.2945704127077743,
 'Thirtis': 0.07081211147380552}

In [47]:
profAiling("War American President")

{'Female': 0.2908390480073644, 'Male': 0.7091609519926356}

In [37]:
app_1 =  gr.Interface(fn = profAiling, 
                    inputs="text", 
                    outputs=gr.outputs.Label(),
                    description="Welcome to profAIling app!\n\nSubmit your post and we will guess if you are a male or a female",
                    examples = ["Rome is really hot in summer... Let's go shopping", "I really love the statistical learning project!", "I want to go on my vacation after a stressing year",])

In [38]:
app_2 = gr.Interface(fn = profAiling_age, 
                    inputs="text", 
                    outputs=gr.outputs.Label(),
                    description="Welcome to profAIling app!\n\nSubmit your post and we will guess your age",
                    examples = ["Rome is really hot in summer... Let's go shopping", "I really love the statistical learning project!", "I want to go on my vacation after a stressing year",])

In [39]:
demo = gr.TabbedInterface([app_1, app_2], ["Gender", "Age"], title = "ProfAIling")

demo.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
